In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from code_utils.read_ipcc_bib import read_bib_wg
from code_utils.enriching_data_OpenAlex import get_countries_concepts_sdg,get_open_alex_data,get_open_alex_data_not_in_references
from code_utils.utils import aplatir,wg_chap_to_dict,get_doi_cleaned,check_doi_glutton,get_year_ipcc
from code_utils.pickle import load_cache,write_cache

In [ ]:
cached_openalex_data = {}
cached_openalex_data_not_ipcc = {}

In [ ]:
try:
    cached_openalex_data = load_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
    cached_openalex_data_not_ipcc = load_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')
except:
    #write_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
    #write_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')
    print('oups not in bibliography folder')

Read IPCC references in .bib files

In [ ]:
wgs={'1':{'wg1':[], 'dataframes_1':{}, 'listdir1':['IPCC_AR6_WGI_References_Chapter01.bib',
 'IPCC_AR6_WGI_References_Chapter02.bib',
 'IPCC_AR6_WGI_References_Chapter03.bib',
 'IPCC_AR6_WGI_References_Chapter04.bib',
 'IPCC_AR6_WGI_References_Chapter05.bib',
 'IPCC_AR6_WGI_References_Chapter06.bib',
 'IPCC_AR6_WGI_References_Chapter07.bib',
 'IPCC_AR6_WGI_References_Chapter08.bib',
 'IPCC_AR6_WGI_References_Chapter09.bib',
 'IPCC_AR6_WGI_References_Chapter10.bib',
 'IPCC_AR6_WGI_References_Chapter11.bib',
 'IPCC_AR6_WGI_References_Chapter12.bib']},
     '2':{'wg2':[], 'wg2_update':[], 'dataframes_2':{}, 'listdir2':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2")},
     '2_CROSS':{'wg2_CROSS':[], 'wg2_CROSS_update':[], 'dataframes_2_CROSS':{}, 'listdir2_CROSS':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG2_CROSS")},
     '3':{'wg3':[], 'dataframes_3':{}, 'listdir3':os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG3")}}

In [ ]:
os.listdir(module_path+f"\\IPCC_bibliography\\AR6\\WG1")

In [ ]:
for k in ['1','2','2_CROSS','3']:
    wgs[k][f'df_{k}'] = read_bib_wg(wgs,k,verbose=True)
    #wgs[k][f'df_{k}'].to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', orient='records', lines=True)

Read and clean the data from 'structured_data' folder

In [ ]:
data_all={}

In [ ]:
for k in ['1','2','2_CROSS','3']:
   #data_all[f'df_wg{k}']=wgs[k][f'df_{k}']
   data_all[f'df_wg{k}']=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_wg{k}.jsonl', lines= True)

In [ ]:
#all data 
df_ipcc=pd.concat(list(data_all.values()), ignore_index=True)

In [ ]:
df_ipcc.loc[:,'doi']=df_ipcc.loc[:,'doi'].apply(lambda x: get_doi_cleaned(x))
df_ipcc.loc[:,'doi']=df_ipcc.loc[:,'doi'].apply(lambda x: None if str(x)[:4]=='http' else x)
df_ipcc['freq']=1

Enriching data through OpenAlex API integration

In [ ]:
#df_unique_doi=df_ipcc.groupby(by=['doi','title','year'], dropna=True).agg({'wg': lambda x: list(x),'chap': lambda x: list(x)})
df_unique_doi=df_ipcc[['doi','title','year']].drop_duplicates(subset=['doi']).dropna(subset=['doi'])
df_unique_doi=df_unique_doi.reset_index()

In [ ]:
#get data_OpenAlex from the API 
df_unique_doi.apply(lambda row: get_open_alex_data(cached_openalex_data,row['doi']), axis= 1)

In [ ]:
len(cached_openalex_data)

In [ ]:
write_cache(cached_openalex_data,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')

In [ ]:
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_list=[]
locations_id_list=[]
type_list=[]
type_crossref_list=[]

for i,row in df_unique_doi.iterrows():
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_id,type_OA,type_crossref=get_countries_concepts_sdg(cached_openalex_data,row)

    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_list.append(locations_names)
    locations_id_list.append(locations_id)
    type_list.append(type_OA)
    type_crossref_list.append(type_crossref)
    

df_unique_doi['countries'] = countries_list
df_unique_doi['concepts'] = concepts_list
df_unique_doi['sdg'] = sdg_list
df_unique_doi['topics'] = topics_list
df_unique_doi['is_OA_available'] = is_OA_available_list
df_unique_doi['title_OA'] = title_list
df_unique_doi['year_OA'] = year_list
df_unique_doi['authors_name'] = names_list
df_unique_doi['rors'] = rors_list
df_unique_doi['institutions_names'] = institutions_names_list
df_unique_doi['locations_names'] = locations_list
df_unique_doi['locations_ids'] = locations_id_list
df_unique_doi['type'] = type_list
df_unique_doi['type_crossref'] = type_crossref_list

In [ ]:
df_unique_doi.locations_ids[0]

In [ ]:
#df_unique_doi['wg_chap']=df_unique_doi.apply(lambda row: wg_chap_to_dict(row), axis=1)
df_unique_doi.loc[:,'countries']=df_unique_doi.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_unique_doi.loc[:,'countries']=df_unique_doi.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)
df_unique_doi.loc[:,'year']=df_unique_doi.loc[:,'year'].apply(lambda x: get_year_ipcc(x))
df_unique_doi['test_glutton']=df_unique_doi.apply(lambda row: check_doi_glutton(row), axis=1)
df_unique_doi=df_unique_doi[(df_unique_doi['test_glutton']==True)&(pd.isna(df_unique_doi['is_OA_available'])==False)]

In [ ]:
df_unique_doi.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', orient='records', lines= True)

In [ ]:
df_ipcc=df_ipcc[['year', 'url', 'doi','title','author','freq']]

In [ ]:
df_ipcc=pd.merge(df_ipcc, df_unique_doi, on='doi', how='left')
df_ipcc.loc[:,'countries']=df_ipcc.loc[:,'countries'].apply(lambda x: str(x).replace('nan','None').replace('[]','None').replace('[None]','None') if ((str(x)=='nan')|(str(x)=='[None]')|((str(x)=='[]'))) else x )
df_ipcc.loc[:,'countries']=df_ipcc.loc[:,'countries'].apply(lambda x: ['None'] if x=='None' else x)

In [ ]:
df_ipcc.loc[:,'year']=df_ipcc.loc[:,'year'].apply(lambda x: get_year_ipcc(x))

In [ ]:
df_ipcc['test_glutton']=df_ipcc.apply(lambda row: check_doi_glutton(row), axis=1)

In [ ]:
df_ipcc.drop_duplicates(subset=['doi']).test_glutton.value_counts()

In [ ]:
df_ipcc=df_ipcc[(df_ipcc['test_glutton']==True)&(pd.isna(df_ipcc['is_OA_available'])==False)]

In [ ]:
df_ipcc.to_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json', orient='records')

Find the data for constructing the learning model ( IPCC related or not )

In [ ]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [ ]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids'])

In [ ]:
len(df_ipcc)

In [ ]:
df_ipcc['year'] = pd.to_numeric(df_ipcc['year'], errors='coerce')
year=df_ipcc.drop_duplicates(subset='doi')['year'].dropna().sort_values().apply(lambda x: int(x))
year_counts = pd.Series(list(year)).value_counts().to_dict()

In [ ]:
sum(list(year_counts.values()))

In [ ]:
len(aplatir(list(cached_openalex_data_not_ipcc.values())))

In [ ]:
dois=list(df_ipcc.doi.dropna().drop_duplicates())+list(df_not_ipcc.doi.dropna().drop_duplicates())
year_counts_not_ipcc={}

In [ ]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipcc[year]=[]
    year_counts_not_ipcc[year]=0
    while year_counts_not_ipcc[year]<year_counts[year]:
        get_open_alex_data_not_in_references(dois,cached_openalex_data_not_ipcc,year_counts,year_counts_not_ipcc,year)
    cached_openalex_data_not_ipcc[year]=cached_openalex_data_not_ipcc[year][:year_counts[year]]

In [ ]:
for year in list(year_counts.keys()):   
    cached_openalex_data_not_ipcc[year]=cached_openalex_data_not_ipcc[year][:year_counts[year]]

In [ ]:
write_cache(cached_openalex_data_not_ipcc,module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_not_ipcc.pkl')

In [ ]:
df_not_ipcc=pd.DataFrame()
countries_list = []
concepts_list = []
sdg_list = []
topics_list = []
is_OA_available_list=[]
title_list=[]
year_list=[]
names_list=[]
rors_list=[]
institutions_names_list=[]
locations_list=[]
doi_list=[]
locations_id_list=[]


for k in range(len(aplatir(list(cached_openalex_data_not_ipcc.values())))):
    countries,concepts,sdg,year,topics,doi,bool_OA,title,name,rors,institutions_names,locations_names,locations_id=get_countries_concepts_sdg(cached_openalex_data=aplatir(list(cached_openalex_data_not_ipcc.values())),ipcc=False,i=k)

    doi_list.append(doi)
    countries_list.append(countries)
    concepts_list.append(concepts)
    sdg_list.append(sdg)
    topics_list.append(topics)
    is_OA_available_list.append(bool_OA)
    title_list.append(title)
    year_list.append(year)
    names_list.append(name)
    rors_list.append(rors)
    institutions_names_list.append(institutions_names)
    locations_list.append(locations_names)
    locations_id_list.append(locations_id)
    
df_not_ipcc['doi'] = doi_list
df_not_ipcc['countries'] = countries_list
df_not_ipcc['concepts'] = concepts_list
df_not_ipcc['sdg'] = sdg_list
df_not_ipcc['topics'] = topics_list
df_not_ipcc['is_OA_available'] = is_OA_available_list
df_not_ipcc['title'] = title_list
df_not_ipcc['year'] = year_list
df_not_ipcc['authors_name'] = names_list
df_not_ipcc['rors'] = rors_list
df_not_ipcc['institutions_names'] = institutions_names_list
df_not_ipcc['locations_names'] = locations_list
df_not_ipcc['locations_ids'] = locations_id_list

In [ ]:
len(df_not_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']))

In [ ]:
len(df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']))

In [ ]:
df_not_ipcc.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_not_ipcc.jsonl', orient='records', lines= True)